In [5]:
import os
import sys
from pyspark.sql import SparkSession, DataFrame # type: ignore
from pyspark.sql.functions import col, last, row_number, to_timestamp, year, avg, monotonically_increasing_id # type: ignore
from pyspark.sql.window import Window # type: ignore

In [2]:
spark = SparkSession.builder.appName("Testing_load_Silver").getOrCreate()
spark.sql("CREATE DATABASE IF NOT EXISTS datalake.silver")

DataFrame[]

In [3]:
silver_tbl_gold = spark.table("datalake.silver.gold")
silver_tbl_gold.createOrReplaceTempView("silver_tbl_silver")

In [4]:
print(silver_tbl_gold.printSchema())
silver_tbl_gold.show(5)

root
 |-- ID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Price: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)

None


+---+----------+-----+-----+-----+-----+
| ID|      Date|Price| Open| High|  Low|
+---+----------+-----+-----+-----+-----+
|  1|1975-01-03|174.7|173.0|175.5|170.5|
|  2|1975-01-06|174.4|172.0|174.5|167.5|
|  3|1975-01-07|173.4|171.0|174.0|168.5|
|  4|1975-01-08|177.4|176.0|180.0|176.0|
|  5|1975-01-09|178.3|179.0|179.1|177.0|
+---+----------+-----+-----+-----+-----+
only showing top 5 rows



In [6]:
silver_tbl_gold_yearly = silver_tbl_gold.withColumn("Year", year("Date"))
silver_tbl_gold_yearly.createOrReplaceTempView("silver_tbl_silver_yearly")

silver_tbl_gold_yearly.show(5)

+---+----------+-----+-----+-----+-----+----+
| ID|      Date|Price| Open| High|  Low|Year|
+---+----------+-----+-----+-----+-----+----+
|  1|1975-01-03|174.7|173.0|175.5|170.5|1975|
|  2|1975-01-06|174.4|172.0|174.5|167.5|1975|
|  3|1975-01-07|173.4|171.0|174.0|168.5|1975|
|  4|1975-01-08|177.4|176.0|180.0|176.0|1975|
|  5|1975-01-09|178.3|179.0|179.1|177.0|1975|
+---+----------+-----+-----+-----+-----+----+
only showing top 5 rows



In [7]:
yearly_averages = silver_tbl_gold_yearly.groupBy("Year").agg(
    avg("Price").alias("Price_avg"),
    avg("Open").alias("Open_avg"),
    avg("High").alias("High_avg"),
    avg("Low").alias("Low_avg")
)
yearly_averages_with_id = yearly_averages.withColumn("ID", monotonically_increasing_id())
yearly_averages_with_id.createOrReplaceTempView("yearly_averages")

yearly_averages_with_id.show(5)

#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGSEGV (0xb) at pc=0x00007fa3b4551f84, pid=1620, tid=1839
#
# JRE version: OpenJDK Runtime Environment (11.0.26+4) (build 11.0.26+4-post-Debian-1deb11u1)
# Java VM: OpenJDK 64-Bit Server VM (11.0.26+4-post-Debian-1deb11u1, mixed mode, sharing, tiered, compressed oops, g1 gc, linux-amd64)
# Problematic frame:
# V  [libjvm.so+0x5bef84]  ClassLoaderData::ChunkedHandleList::oops_do(OopClosure*) [clone .constprop.0]+0x64
#
# Core dump will be written. Default location: Core dumps may be processed with "/wsl-capture-crash %t %E %p %s" (or dumping to /src/notebooks/core.1620)
#
# An error report file with more information is saved as:
# /src/notebooks/hs_err_pid1620.log


#
# If you would like to submit a bug report, please visit:
#   https://bugs.debian.org/openjdk-11
#


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o96.showString

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:


# Reorder columns with ID first, then Year
result_df = yearly_averages_with_id.select("ID", "Year", "Price_avg", "Open_avg", "High_avg", "Low_avg")

# Show the result
result_df.show()

In [9]:
spark.stop()